# Backtest 2019

## What is the backtest?

The backtest analyzes data from a past period of time to determine the ideal values of seven different metrics (discussed later) for each sector.  This version of the backtest uses the jupyter notebook format to describe each part, as well as to increase efficiency.

#  
### Library Imports

- pandas - used to manipulate data within different data structures
- datetime - used to get today's date and current time
- time - used for calculating runtime
- numpy - supports mathematical functions for large volume data structures
- product - creates a cartesian product

In [6]:
from pandas import *
import pandas as pd
from datetime import *
import time
import numpy as np
from itertools import product
from IPython.display import display

### Effiency and timing

In the past, it has taken dozens of hours to complete the backtests.  This is costly in terms of time and electricity (which should be considered as a sustainability-oriented organization).  The block below assigns the current time to a variable so that we can later determine runtime.

We also establish our sectors and metrics in lists.

In [7]:
st = time.time()

sectors = ['Consumer Discretionary', 'Consumer Staples', 'Energy', 'Financials', 'Health Care', 'Industrials', \
           'Information Technology', 'Materials', 'Real Estate', 'Telecommunication Services', 'Utilities', 'Materials']
metrics = [ 'PE_RATIO', 'PX_TO_BOOK_RATIO', 'TRAIL_12M_EPS', 'TOT_DEBT_TO_TOT_EQY', \
               'PRICE_TO_FCF', 'RETURN_COM_EQY', 'RETURN_ON_ASSET' ]

pd.options.mode.chained_assignment = None #allow chained assignment

### Risk-free rates

In [9]:
#Define risk-free rates
TBill3Mth = (2.34/100)
Libor3Mth = (2.62/100)

#  
#   
# Step 1: Data input and Setup, Organic Range Generation

The basis of our backtest are frames - sets of each metric, each containing a different combination of values.  In order to determine what these values are, we have to come up with a range for each metric, for each sector.

We also import our data file for range generation -- the last 10 years of data for each company.  The data is imported to a **dataframe**, a data structure which can be thought of as an excel sheet manipulable through python.  The dataframe is then sorted.

### Data Import

In [17]:
#sdf = pd.read_excel("data/spyx2007-2017 dat.xlsx", index_col=[0,1])
#^The above code links to the .xlsx file that has QYears instead of dates;
#I'm unsure of the status of QYear, I don't think Bob every fully implemented it.
sdf = read_csv("Backtest VALUES.csv",index_col='DATE',parse_dates=True)

sdf = sdf.sort_index()

#drop any rows without a date
sdf.dropna(axis='index',how='any',inplace=True)

#Define starting date to begin df with
startDate = to_datetime('2014-03-28')
startDate=startDate.toordinal()

#Cut spyx_Sector based on DateTime range
sdf['ordinalTime']=sdf.index
sdf['ordinalTime'] = sdf['ordinalTime'].apply(date.toordinal)
sdf = sdf.loc[sdf['ordinalTime']>=startDate]

### getRanges method

The getRanges method returns a list of ranges, one for each metric, for that specific sector.  A range has a low value, high value, and step value (incrementation value).  While this code is visually unappealing, it's relatively practical. 

It is worth noting that we take ranges from the 25th percentile to the 75th percentile (the middle section of a normal distribution).  The companies in the top 25% for a metric may be outliers or overvalued based on that number, and the companies in the lower 25% may be underperforming.  

It is also worth noting the step values.  These are generally safe bets for typical output to create a reasonable number of frames.  Remember, runtime is **_exponential_**.*

_\*Proposed change: change step-by amounts in range functions to be computer-generated\*_

In [16]:
def getRanges(sector):
    peRange =  range(int(round(np.percentile(sdf.loc[(slice(None),sector), 'PE_RATIO'].dropna(), 25))), \
                     int(round(np.percentile(sdf.loc[(slice(None),sector), 'PE_RATIO'].dropna(), 75))), 2)
    pbRange =  range(int(round(np.percentile(sdf.loc[(slice(None),sector), 'PX_TO_BOOK_RATIO'].dropna(), 25))), \
                     int(round(np.percentile(sdf.loc[(slice(None),sector), 'PX_TO_BOOK_RATIO'].dropna(), 75))))
    epsRange =  range(int(round(np.percentile(sdf.loc[(slice(None),sector), 'TRAIL_12M_EPS'].dropna(), 25))), \
                      int(round(np.percentile(sdf.loc[(slice(None),sector), 'TRAIL_12M_EPS'].dropna(), 75))), 2)
    deRange =  range(int(round(np.percentile(sdf.loc[(slice(None),sector), 'TOT_DEBT_TO_TOT_EQY'].dropna(), 25))), \
                     int(round(np.percentile(sdf.loc[(slice(None),sector), 'TOT_DEBT_TO_TOT_EQY'].dropna(), 75))), 10)
    fcfRange =  range(int(round(np.percentile(sdf.loc[(slice(None),sector), 'PRICE_TO_FCF'].dropna(), 25))), \
                     int(round(np.percentile(sdf.loc[(slice(None),sector), 'PRICE_TO_FCF'].dropna(), 75))), 5)
    roeRange =  range(int(round(np.percentile(sdf.loc[(slice(None),sector), 'RETURN_COM_EQY'].dropna(), 25))), \
                     int(round(np.percentile(sdf.loc[(slice(None),sector), 'RETURN_COM_EQY'].dropna(), 75))), 2)
    roaRange =  range(int(round(np.percentile(sdf.loc[(slice(None),sector), 'RETURN_ON_ASSET'].dropna(), 25))), \
                     int(round(np.percentile(sdf.loc[(slice(None),sector), 'RETURN_ON_ASSET'].dropna(), 75))), 2)
    return [peRange, pbRange, epsRange, deRange, fcfRange, roeRange, roaRange]

#  
#  
# Step 2: Frame Generation and Product

### Output functions

In [ ]:
#Function to output every indivdual portfolio inside each period
def outputIndPeriod(df, date):
        df.to_csv(sector+"IndPortPeriod"+time.strftime("d%dm%my%Y")+".csv", mode='a')
        
#output each portfolio with dates
def outputIndPort(dfPort,params):
    dfPort.to_csv(sector+"IndividualPortfolios"+time.strftime("d%dm%my%Yh")+".csv", \
                  mode='a',index_label=("T"+str(params[0])+str(params[1])+str(params[2])+str(params[3])\
                        +str(params[4])+str(params[5])+str(params[6])))

#Output total portfolio with parameters
def outputTotalPort(dfTotalPort,params):
    dfTotalPort.to_csv(sector+"TotalPortfolio"+time.strftime("d%dm%my%Yh")+".csv", mode='w',index_label=("T"+"Portfolio"))
    
#Function to loop through every date and output an individual portfolio
def calulatePortLoop(dateList, params):
    for date in datesList:
        df = spyx_Sector.loc[spyx_Sector.index==date]
        df['peTest']=params[0]
        df['pbTest']=params[1]
        df['epsTest']=params[2]
        df['deTest']=params[3]
        df['fcfTest']=params[4]
        df['roeTest']=params[5]
        df['roaTest']=params[6]
        df['peScore']=df['PE_RATIO']<params[0]
        df['pbScore']=df['PX_TO_BOOK_RATIO']<params[1]
        df['epsScore']=df['TRAIL_12M_EPS']>params[2]
        df['deScore']=df['TOT_DEBT_TO_TOT_EQY']<params[3]
        df['fcfScore']=df['PRICE_TO_FCF']>params[4]
        df['roeScore']=df['RETURN_COM_EQY']>params[5]
        df['roaScore']=df['RETURN_ON_ASSET']>params[6]
        df['betaScore']=1/(df['ADJUSTED_BETA']*1000)
        df['hsfScore']=df['peScore'].astype(int)+df['pbScore'].astype(int)+df['epsScore'].astype(int)+\
            df['deScore'].astype(int)+df['fcfScore'].astype(int)+df['roeScore'].astype(int)+df['roaScore'].astype(int)
        df['betaScore']
        df['ind']="T"+df['peTest'].map(str)+df['pbTest'].map(str)+df['epsTest'].map(str)+df['deTest'].map(str)+\
            df['fcfTest'].map(str)+df['roeTest'].map(str)+df['roaTest'].map(str)
        df.sort_values(by='hsfScore',ascending=False,inplace=True)
        df=df.iloc[0:5]
        outputIndPeriod(df,date)
        dfPort.loc[dfPort.index==date,'peTest'] = df.ix[0,'peTest']
        dfPort.loc[dfPort.index==date,'pbTest'] = df.ix[0,'pbTest']
        dfPort.loc[dfPort.index==date,'epsTest'] = df.ix[0,'epsTest']
        dfPort.loc[dfPort.index==date,'deTest'] = df.ix[0,'deTest']
        dfPort.loc[dfPort.index==date,'fcfTest'] = df.ix[0,'fcfTest']
        dfPort.loc[dfPort.index==date,'roeTest'] = df.ix[0,'roeTest']
        dfPort.loc[dfPort.index==date,'roaTest'] = df.ix[0,'roaTest']
        dfPort.loc[dfPort.index==date,'maxHSFScore'] = df['hsfScore'].max()
        dfPort.loc[dfPort.index==date,'return'] = (df['RETURN'].mean())
        dfPort.loc[dfPort.index==date,'beta'] = (df['ADJUSTED_BETA'].mean())
    dfPort['treynor']=(dfPort['return']-TBill3Mth)/dfPort['beta']
    dfPort['ind']="T"+dfPort['peTest'].map(str)+dfPort['pbTest'].map(str)+dfPort['epsTest'].map(str)+dfPort['deTest'].map(str)+\
        dfPort['fcfTest'].map(str)+dfPort['roeTest'].map(str)+dfPort['roaTest'].map(str)
    return dfPort

### Create Output Dataframe

In [18]:
#Create set of dates
datesList = sorted(set(sdf.index))

#Create new Output DF for portfolios for dates
dfPort = pd.DataFrame([],index=datesList)
dfPort['return']=None
dfPort['beta']=None
dfPort['tBill3Mth']=TBill3Mth
dfPort['treynor']=None
dfPort['peTest']=None
dfPort['pbTest']=None
dfPort['epsTest']=None
dfPort['deTest']=None
dfPort['fcfTest']=None
dfPort['roeTest']=None
dfPort['roaTest']=None
dfPort['maxHSFScore']=None

#Create new Output DF for total portfolios
dfTotalPort = pd.DataFrame([])
dfTotalPort['ind']=list(product(peRange,pbRange,epsRange,deRange,fcfRange,roeRange,roaRange))
dfTotalPort['meanReturn']=None
dfTotalPort['meanBeta']=None
dfTotalPort['tBill3Mth']=TBill3Mth
dfTotalPort['totalTreynor']=None
dfTotalPort['peTest']=None
dfTotalPort['pbTest']=None
dfTotalPort['epsTest']=None
dfTotalPort['deTest']=None
dfTotalPort['fcfTest']=None
dfTotalPort['roeTest']=None
dfTotalPort['roaTest']=None

### Cartesian Product

In [ ]:
for params in product(peRange, pbRange, epsRange, deRange, fcfRange, roeRange, roaRange):
    calulatePortLoop(datesList, params)
    dfTotalPort.loc[dfTotalPort['ind']==params,'meanReturn'] = dfPort['return'].mean()
    dfTotalPort.loc[dfTotalPort['ind']==params,'meanBeta'] = dfPort['beta'].mean()
    dfTotalPort.loc[dfTotalPort['ind']==params,'peTest'] = params[0]
    dfTotalPort.loc[dfTotalPort['ind']==params,'pbTest'] = params[1]
    dfTotalPort.loc[dfTotalPort['ind']==params,'epsTest'] = params[2]
    dfTotalPort.loc[dfTotalPort['ind']==params,'deTest'] = params[3]
    dfTotalPort.loc[dfTotalPort['ind']==params,'fcfTest'] = params[4]
    dfTotalPort.loc[dfTotalPort['ind']==params,'roeTest'] = params[5]
    dfTotalPort.loc[dfTotalPort['ind']==params,'roaTest'] = params[6]
    print(dfPort)
    outputIndPort(dfPort,params)

### Treynor Calculations, Output

In [ ]:
dfTotalPort['totalTreynor']=(dfTotalPort['meanReturn']-dfTotalPort['tBill3Mth'])/dfTotalPort['meanBeta']
dfTotalPort.sort_values('totalTreynor',ascending=False, inplace=True)
dfTotalPort.set_index("T"+dfTotalPort['peTest'].map(str) + dfTotalPort['pbTest'].map(str) +\
                      dfTotalPort['epsTest'].map(str) + dfTotalPort['deTest'].map(str) +\
                      dfTotalPort['fcfTest'].map(str) + dfTotalPort['roeTest'].map(str) +\
                      dfTotalPort['roaTest'].map(str), inplace = True)
outputTotalPort(dfTotalPort,params)

## Step 3: Output

### Runtime

The following displays the total runtime

In [45]:
import datetime
et = time.time()
str(datetime.timedelta(seconds = et - st))

'0:01:21.227196'